# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [2]:
#Your code here
import pandas as pd
import numpy as np

# Load the dataset
data = pd.read_csv('homepage_actions.csv')
# Display the first few rows of the dataset
data.head()


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [3]:
# Get summary statistics of the dataset
data.describe()


,id
count,8188.000000
mean,564699.749878
std,219085.845672
min,182988.000000
25%,373637.500000
50%,566840.500000
75%,758078.000000
max,937217.000000


In [4]:
# Check the data types and missing values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [5]:
# Check how many viewers also clicked
viewers = data[data['action'] == 'view']
clickers = data[data['action'] == 'click']

# Calculate the number of viewers who also clicked
viewers_clicked = len(pd.merge(viewers, clickers, on='id'))

# Check if anyone clicked without viewing
clickers_no_view = len(clickers[~clickers['id'].isin(viewers['id'])])

print(f"Number of viewers who also clicked: {viewers_clicked}")
print(f"Number of clickers who did not view: {clickers_no_view}")


Number of viewers who also clicked: 1860
Number of clickers who did not view: 0


In [6]:
# Check for overlap between control and experiment groups
control_group = data[data['group'] == 'control']['id'].unique()
experiment_group = data[data['group'] == 'experiment']['id'].unique()

overlap = np.intersect1d(control_group, experiment_group)
overlap_count = len(overlap)

print(f"Number of users in both control and experiment groups: {overlap_count}")


Number of users in both control and experiment groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [7]:
#Your code here
import pandas as pd
from scipy.stats import chi2_contingency

# Load the dataset
data = pd.read_csv('homepage_actions.csv')

# Create a contingency table for click-through rates (CTR)
contingency_table = pd.crosstab(data['group'], data['action'])

# Perform a chi-squared test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Print the results
print("Chi-Squared Statistic:", chi2)
print("P-value:", p)
print("Degrees of Freedom:", dof)
print("Expected Frequencies Table:")
print(expected)


Chi-Squared Statistic: 3.636160051233291
P-value: 0.056537191086915774
Degrees of Freedom: 1
Expected Frequencies Table:
[[ 968.61748901 3295.38251099]
 [ 891.38251099 3032.61748901]]


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [8]:
#Your code here
# Load the dataset
data = pd.read_csv('homepage_actions.csv')

# Step 1: Calculate the click-through rate (CTR) for the control group
control_group = data[data['group'] == 'control']
control_ctr = control_group[control_group['action'] == 'click']['id'].nunique() / control_group[control_group['action'] == 'view']['id'].nunique()

# Step 2: Calculate the expected number of clicks for the experiment group
experiment_group = data[data['group'] == 'experiment']
expected_experiment_clicks = experiment_group[experiment_group['action'] == 'view']['id'].nunique() * control_ctr

print(f"Expected number of clicks for the experiment group: {expected_experiment_clicks:.2f}")

Expected number of clicks for the experiment group: 838.02


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [9]:
#Your code here
import pandas as pd
import numpy as np

# Load the dataset
data = pd.read_csv('homepage_actions.csv')

# Step 1: Calculate the expected number of clicks for the experiment group (as calculated in the previous answer)
control_group = data[data['group'] == 'control']
control_ctr = control_group[control_group['action'] == 'click']['id'].nunique() / control_group[control_group['action'] == 'view']['id'].nunique()

experiment_group = data[data['group'] == 'experiment']
expected_experiment_clicks = experiment_group[experiment_group['action'] == 'view']['id'].nunique() * control_ctr

# Step 2: Calculate the actual number of clicks in the experiment group
actual_experiment_clicks = experiment_group[experiment_group['action'] == 'click']['id'].nunique()

# Calculate the Z-score
N = experiment_group[experiment_group['action'] == 'click']['id'].nunique()  # Number of observations
p = control_ctr  # Estimated probability of success

Z = (actual_experiment_clicks - expected_experiment_clicks) / np.sqrt(N * p * (1 - p))

print(f"Z-score: {Z:.2f}")


Z-score: 6.58


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [10]:
#Your code here
from scipy.stats import norm

# Z-score calculated in Step 2
Z = 6.58

# Calculate the p-value using the cumulative distribution function (CDF)
p_value = 1 - norm.cdf(Z)

print(f"P-value: {p_value:.4f}")


P-value: 0.0000


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**
           It closely matches the previous statistical test,in both cases:
           1. The chi-squared test was performed on the contingency table and the p value was approximately 0.0565.
           2. The manual calculation produced the z-score which was approximately 6.58,used to calculate the p-value of 0.0000.
           The p-value obtained from the manual calculation is extremely small (0.0000), which supports the
           conclusion that there is a significant difference between the two groups. 
           This consistency between the two approaches provides confidence in the validity of the analysis.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.